# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [1]:
import xlwings as xw
import numpy as np
import pandas as pd

book1 = xw.Book("себестоимостьА_в1.xlsx")
sheet = book1.sheets["Рецептура"]
consumption = sheet.range("G7:O10").options(np.array).value
price_per_unit = sheet.range("G14:O14").options(np.array).value
cost_price = consumption * price_per_unit
cost_price[np.isnan(cost_price)] = 0
cost_price = np.round(np.sum(cost_price, axis=1).tolist(), 2)
cost_price

array([21.48, 16.52, 17.42, 18.08])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [2]:
sheet.range("T7:T10").options(transpose=True).value = cost_price
sheet.range("T6").value = "Себестоимость"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [3]:
sheet.range("T4:T6").column_width = 15 
sheet.range("T4:T6").color = 255,192,0 
sheet.range("T4:T6").api.Font.ColorIndex = 2   
sheet.range("T4:T6").api.Font.Size = 11
sheet.range("T4:T10").api.Borders.Weight = 2
sheet.range("T4:T6").api.HorizontalAlignment = -4108    # -4108 Horizontally centered. -4131 is on the left, -4152 is on the right.
sheet.range("T4:T6").api.VerticalAlignment = -4108
sheet.range("T7:T10").color = 255,255,167 
sheet.range("T4:T6").merge()



4. Выполнить 3 с помощью "протягиваемых" формул.

In [4]:
sheet.range("U7:U10").formula = "=SUMPRODUCT(G7:O7, $G$14:$O$14)"

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [5]:
import pandas as pd

# Загрузка данных из файла reviews_sample.csv
reviews = pd.read_csv('reviews_sample.csv', delimiter=',')

# Загрузка данных из файла recipes_sample.csv
recipes = pd.read_csv('recipes_sample.csv', delimiter= ',')

# Оставляем только нужные столбцы в таблице с рецептами
recipes = recipes[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]

# Вывод результатов
print(reviews.head())
print(recipes.head())


   Unnamed: 0     user_id  recipe_id        date  rating  \
0      370476       21752      57993  2003-05-01       5   
1      624300      431813     142201  2007-09-16       5   
2      187037      400708     252013  2008-01-10       4   
3      706134  2001852463     404716  2017-12-11       5   
4      312179       95810     129396  2008-03-14       5   

                                              review  
0  Last week whole sides of frozen salmon fillet ...  
1  So simple and so tasty!  I used a yellow capsi...  
2  Very nice breakfast HH, easy to make and yummy...  
3  These are a favorite for the holidays and so e...  
4  Excellent soup!  The tomato flavor is just gre...  
      id                                      name  minutes   submitted  \
0  44123     george s at the cove  black bean soup       90  2002-10-25   
1  67664        healthy for them  yogurt popsicles       10  2003-07-26   
2  38798              i can t believe it s spinach       30  2002-08-29   
3  35173 

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [6]:
import pandas as pd

# Загрузка данных из файла reviews_sample.csv
reviews = pd.read_csv('reviews_sample.csv', index_col=0, encoding='utf-8')

# Загрузка данных из файла recipes_sample.csv
recipes = pd.read_csv('recipes_sample.csv', index_col=0, encoding='utf-8')

# Выбор случайных 5% строк из каждой таблицы
reviews_sample = reviews.sample(frac=0.05)
recipes_sample = recipes.sample(frac=0.05)

# Создание объекта ExcelWriter для записи в файл
writer = pd.ExcelWriter('recipes.xlsx')

# Запись таблиц на разные листы в файл
reviews_sample.to_excel(writer, sheet_name='Отзывы', index=False)
recipes_sample.to_excel(writer, sheet_name='Рецепты', index=False)

# Сохранение файла
writer.save()

# Чтение сохраненных данных из файла Excel и вывод на экран
df_reviews = pd.read_excel('recipes.xlsx', sheet_name='Отзывы')
df_recipes = pd.read_excel('recipes.xlsx', sheet_name='Рецепты')

print("Таблица Отзывы:\n", df_reviews)
print("Таблица Рецепты:\n", df_recipes)


C:\Users\Артём\AppData\Local\Temp\ipykernel_17672\1494469552.py:21: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Таблица Отзывы:
          user_id  recipe_id        date  rating  \
0         829870      78897  2008-11-19       5   
1         579557     125767  2007-09-13       5   
2         126440     322668  2014-03-13       5   
3         368078      42038  2009-08-30       5   
4         141042      21688  2004-10-08       5   
...          ...        ...         ...     ...   
6330      205832      83411  2007-04-25       5   
6331      125388     176519  2009-11-01       5   
6332  1802849661     374725  2016-04-04       5   
6333     1055589     117641  2008-12-05       5   
6334      387646     202027  2007-03-08       5   

                                                 review  
0                                   Great quick recipe.  
1     This was EXACTLY what I was hoping for!  I als...  
2     Made fresh pesto and ground my chicken so this...  
3     Yumm!!!  This was excellent!  I used 1 Cup fro...  
4     A great casserole, just delicious.  This dish ...  
...                   

In [7]:
import xlwings as xw

#Открываем книгу Excel с названием 'recipes.xlsx' с помощью xlwings
recipes_ex = xw.Book('recipes.xlsx')

#Выбираем лист 'Рецепты'
recipes_sheet = recipes_ex.sheets['Рецепты']

#Добавляем заголовок для нового столбца в ячейку H1
recipes_sheet.range("H1").value = 'seconds_assign'

#Вычисляем время выполнения рецепта в секундах и сохраняем его в массив seconds_assign
second_assign = (recipes_sample["minutes"] * 60).to_numpy()

#Присваиваем массив значений столбцу H в Excel-файле
recipes_sheet.range("H2:H1501").options(transpose=True).value = second_assign

#Выводим значения столбца H для проверки
print(recipes_sheet.range("H2:H1501").value)

[1200.0, 1800.0, 2700.0, 2700.0, 1500.0, 2700.0, 1200.0, 1200.0, 2100.0, 600.0, 3600.0, 2400.0, 900.0, 300.0, 240.0, 1500.0, 3600.0, 29700.0, 1560.0, 1200.0, 3000.0, 2400.0, 300.0, 1080.0, 1800.0, 600.0, 3300.0, 3300.0, 180.0, 4800.0, 3600.0, 600.0, 1620.0, 1200.0, 300.0, 2700.0, 900.0, 600.0, 22200.0, 22800.0, 3480.0, 900.0, 3600.0, 1980.0, 1800.0, 1320.0, 14400.0, 300.0, 2100.0, 35100.0, 29700.0, 3000.0, 1320.0, 4200.0, 2400.0, 300.0, 900.0, 600.0, 1800.0, 6000.0, 2400.0, 2160.0, 1200.0, 2400.0, 1800.0, 1800.0, 18600.0, 1500.0, 4800.0, 2880.0, 960.0, 900.0, 1560.0, 13200.0, 1500.0, 1800.0, 1800.0, 600.0, 600.0, 5400.0, 2700.0, 2400.0, 4800.0, 2700.0, 2400.0, 900.0, 1200.0, 900.0, 900.0, 2400.0, 9900.0, 3000.0, 2100.0, 2100.0, 7500.0, 43800.0, 2100.0, 3900.0, 1200.0, 300.0, 3000.0, 7200.0, 3300.0, 1320.0, 4500.0, 14400.0, 1200.0, 4500.0, 2100.0, 4500.0, 1200.0, 8700.0, 300.0, 1200.0, 600.0, 1620.0, 1800.0, 1500.0, 9600.0, 1200.0, 3900.0, 1200.0, 3000.0, 2400.0, 2400.0, 600.0, 6900.0, 

In [ ]:
import xlwings as xw

# Открываем файл
wb = xw.Book('recipes.xlsx')
# Выбираем лист Рецепты
ws = wb.sheets['Рецепты']

ws.range('I1').value = 'seconds_formula'


# Добавляем формулу Excel в столбец seconds_formula
ws.range('I2').formula = '=TIMEVALUE("00:"&B2)*86400'
last_row = ws.range('A1').current_region.last_cell.row
ws.range('I2:I{}'.format(last_row)).formula = '=TIMEVALUE("00:"&B2)*86400'
ws.range('I2:I{}'.format(last_row)).api.Copy()
ws.range('I3:I{}'.format(last_row)).api.PasteSpecial(-4122)
ws.range('I2').api.CutCopyMode = False

# Сохраняем файл
wb.save()



5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
recipes_sheet.range('H1').api.Font.Bold = True
recipes_sheet.range('I1').api.Font.Bold = True
recipes_sheet.range('H1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
recipes_sheet.range('I1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
import xlwings as xw

# Определение цветов для раскраски
green = (0, 255, 0)
yellow = (255, 255, 0)
red = (255, 0, 0)

# Получение столбца минут
minutes_range = recipes_sheet.range('B2:B1501')

# Применение условного форматирования
for cell in minutes_range:
    minutes = cell.value
    if minutes < 5:
        color = green
    elif minutes < 10:
        color = yellow
    else:
        color = red
    cell.color = color

# Сохранение изменений в файле
wb.save()

In [ ]:
import xlwings as xw

# Открываем файл
wb = xw.Book('recipes.xlsx')
# Выбираем лист Рецепты
ws = wb.sheets['Рецепты']

# Добавляем заголовок столбца
ws.range('I1').value = 'n_reviews'

# Записываем формулу в первую ячейку нового столбца
ws.range('I2').formula = '=COUNTIF(C2:C1501, "<>")'

# Расширяем формулу до конца столбца
ws.range('I2').expand('down').formula = ws.range('I2').formula

# Сохраняем файл
wb.save()



## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate():
    # получаем доступ к листу Отзывы и Рецепты
    reviews_sheet = wb.sheets['Отзывы']
    recipes_sheet = wb.sheets['Рецепты']
    
    # получаем значения столбцов Рейтинг и Рецепт
    ratings = reviews_sheet.range('B2:B1501').value
    recipes = reviews_sheet.range('C2:C1501').value
    
    # проходимся по каждой строке и проверяем соответствие правилам
    for i in range(len(ratings)):
        # проверяем рейтинг
        if not (0 <= ratings[i] <= 5):
            reviews_sheet.range(f'B{i+2}:C{i+2}').color = (255, 0, 0)
        # проверяем рецепт
        if recipes[i] not in recipes_sheet.range('A2:A1501').value:
            reviews_sheet.range(f'B{i+2}:C{i+2}').color = (255, 0, 0)


9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
import csv
import xlwings as xw

# Считываем данные из csv-файла
with open('recipes_model.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter="\t")
    data = list(reader)

# Создаем новую книгу в Excel
wb = xw.Book()
ws = wb.sheets.add('Модель')

# Записываем данные на лист Модель
ws.range('A2:I18').value = data

# Сохраняем книгу
wb.save('recipes_model.xlsx')

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
import xlwings as xw

# Открываем файл
wb = xw.Book('recipes_model.xlsx')

# Выбираем лист
ws = wb.sheets['Модель']

# Определяем последнюю строку с данными
last_row = ws.range('A' + str(ws.cells.last_cell.row)).end('up').row

# Добавляем формулу в столбец J
formula = '=B2&" "&UPPER(C2)'
ws.range('J2').value = formula

# Копируем формулу до последней строки с данными
ws.range('J2').api.autofill(ws.range(f'J2:J{last_row}').api, 0)

# Протягиваем форматирование до последней строки
ws.range('J2:J' + str(last_row)).api.autofit()

# Сохраняем и закрываем файл
wb.save()
wb.close()


11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
import xlwings as xw

# открываем книгу и выбираем лист
wb = xw.Book('recipes_model.xlsx')
sht = wb.sheets['Модель']

# выбираем заголовки и стилизуем их
headers = sht.range('A1:J1')
headers.color = (0, 204, 255) # заливка цветом
headers.api.Font.Bold = True # полужирный шрифт

# применяем автоподбор ширины столбца
sht.autofit('c')

# добавляем таблице автофильтр
sht.range('A1:J1').api.AutoFilter()

# сохраняем изменения
wb.save()



12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 

In [ ]:
import pandas as pd
import xlwings as xw

# подключаемся к книге Excel
wb = xw.Book("recipes_model.xlsx")

# считываем данные из файла recipes_model.csv
data = pd.read_csv("recipes_model.csv")

# группируем данные по сущностям и подсчитываем количество атрибутов
stats = data.groupby("Сущность")["Атрибут"].agg("count")

# создаем новый лист "Статистика" и записываем результат группировки
wb.sheets.add("Статистика").range("A1").value = stats

# строим столбчатую диаграмму и сохраняем ее на лист "Статистика"
stats.plot(kind="bar")
wb.sheets["Статистика"].pictures.add(stats.plot().figure, name="Статистика", update=True, left=wb.sheets["Статистика"].range("E2").left, top=wb.sheets["Статистика"].range("E2").top)

# сохраняем и закрываем книгу Excel
wb.save()
wb.close()
